In [1]:
from datetime import datetime as dt
import numpy as np
import pandas as pd
import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
from dash.dependencies import Input, Output, State

## 1. Carga de datos

In [2]:
columns = ['fpId', 'vectorId', 'latitude', 'longitude', 'altitude', 'aerodromeOfDeparture', 'timestamp', 'RTA',
       'departureDelay', 'flightDate', 'hav_distance', 'speed', 'vspeed']
# 'wind_dir_degrees', 'wind_speed_kt',
#        'visibility_statute_mi', 'wx_string', 'sky_condition', 'temperature',
#        'day_of_week', 'time_of_day',

In [3]:
df_spain = pd.read_parquet('./data/train/train.202001.parquet', columns=columns).sort_values(['fpId', 'timestamp'])
df_spain['timestamp'] = pd.to_datetime(df_spain.timestamp).astype('int64') /10**9
df_spain['flightDate'] = df_spain.flightDate.astype(str)
display(df_spain.head(3))

,fpId,vectorId,latitude,longitude,altitude,aerodromeOfDeparture,timestamp,RTA,departureDelay,flightDate,hav_distance,speed,vspeed
0,AT02685437,20200101-471EFF-000056,48.1134,16.5622,50,LOWW,1.577855,8849,-360,2020-01-01,1121.901007,101.080002,0
1,AT02685437,20200101-471EFF-000057,48.1137,16.5614,50,LOWW,1.577855,8848,-360,2020-01-01,1121.873757,124.779999,65
2,AT02685437,20200101-471EFF-000058,48.1161,16.5540,50,LOWW,1.577855,8838,-360,2020-01-01,1121.612633,143.550003,65


## 2. Definición del dashboard

### 2.1. Inicialización

In [4]:
# Estilos de los ejemplos de Dash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

### 2.2. Definición de elementos de interacción

In [5]:
# Elementos del panel lateral

# Declaración de elementos y construcción del layout
style_dd = dict(minWidth = 150, width='100%', maxWidth=300, fontSize='95%')
style_dp = dict(minWidth = 150, marginLeft=100)
style_lb = {'margin':10}
style_tab = dict(height=30, paddingTop=3)

# Selector de fecha
sel_fecha = dcc.DatePickerSingle(
    id = 'sel_fecha',
    placeholder = 'Selecciona...',
    date=df_spain.flightDate.min(),
    initial_visible_month=df_spain.flightDate.min(),
    min_date_allowed = df_spain.flightDate.min(),
    max_date_allowed = df_spain.flightDate.max(),
    style=style_dp
)
# Lista de opciones de aeropuerto de origen
dd_origen = dcc.Dropdown(
    id = 'dd_origen',
    options = [dict(label=x, value=x) for x in sorted(df_spain.aerodromeOfDeparture.unique())],
    style = style_dd,
    placeholder='Selecciona...',
    multi=False
)
# Lista de opciones de trayectorias disponibles
# - Si no se ha filtrado, se muestran todas las del día seleccionado
# - Si se ha filtrado, se muestran las trayectorias que cumplen todas las condiciones
dd_trayectoria = dcc.Dropdown(
    id = 'dd_trayectoria',
    style = style_dd,
    placeholder='Selecciona...',
    multi=True
)

In [6]:
### Elementos de las pestañas

# Lista de opciones de tipos de mapas en los que mostrar la trayectoria
dd_tipo_mapa = dcc.Dropdown(
    id = 'dd_tipo_mapa',
    style = style_dd,
    placeholder='Selecciona...',
    multi=False,
    options=[{'label':'Mapa 3D', 'value':'3d'},
             {'label':'Simplificado', 'value':'geo'},
             {'label':'Mapbox', 'value':'mapbox'}],
    value = 'mapbox'
)

dd_caracteristica = dcc.Dropdown(
    id = 'dd_caracteristica',
    style = style_dd,
    placeholder='Selecciona...',
    multi=False,
    options=[{'label':'Altitud', 'value':'altitude'},
           {'label':'Velocidad', 'value':'speed'},
           {'label':'Velocidad vertical', 'value':'vspeed'}],
    value = 'altitude'
)

### 2.3. Construcción del layout principal del dashboard

In [7]:
# Construcción directa del layout
app.layout = html.Div(children=[
    html.H1(children='Dashboard interactivo'),
    html.Div(children=[
        html.Div(id='div_filtro',children=[
            html.Label(['Fecha', sel_fecha],
                       style=style_lb),
            html.Label(['Aeropuerto de origen', dd_origen],
                       style=style_lb),
            
            html.Label(['Trayectoria', dd_trayectoria],
                       style=style_lb),
            html.Hr(), 
            dcc.Tabs(id="tabs", value='mapas',  children=[
                dcc.Tab(label='Visor', value='visor', style = style_tab, selected_style = style_tab),
                dcc.Tab(label='Perfiles', value='perfiles', style = style_tab, selected_style = style_tab),
                dcc.Tab(label='Mapas', value='mapas', style = style_tab, selected_style = style_tab)          
            ]),
        ],style = dict(width='18%',minWidth=250,display='inline-block',
                       borderWidth=1,borderStyle='solid', borderRadius=10)), 
        html.Div(children=[
            
            html.Div(id='contenidos-tabs', style=dict(width='100%'))
        ], style = dict(width='80%', display='inline-block', verticalAlign='top',padding='0px 10px 10px 10px'))
    ])
], style=dict(height='100%'))

### 2.4. Definición de callbacks

In [8]:
# Cargando los datos desde los CSV
# Desactivar la celda si se usa HBase

@app.callback(
    [Output(component_id='dd_origen', component_property='options'),
     Output(component_id='dd_trayectoria', component_property='options'),
     Output(component_id='dd_origen', component_property='value')],
    [Input(component_id='sel_fecha', component_property='date')]
)
def introducir_fecha(fecha):
    df = df_spain[df_spain.flightDate == fecha]
    return ([{'label':'Todos', 'value':'todos'}] + [{'label':x, 'value':x} for x in sorted(df.aerodromeOfDeparture.unique())],
            [{'label':x, 'value':x} for x in df.fpId.unique()],
           'todos')

@app.callback(
    [Output(component_id='dd_trayectoria', component_property='options')],
    [Input(component_id='dd_origen', component_property='value')],
    [State(component_id='sel_fecha', component_property='date')]
)
def filtrar_trayectorias(origen, fecha):
    df = df_spain[(df_spain.flightDate == fecha) & (df_spain.aerodromeOfDeparture == origen)]

    return ([{'label':x, 'value':x} for x in df.fpId.unique()],)



@app.callback(
    Output(component_id='grafico_perfiles', component_property='figure'),
    [Input(component_id='dd_caracteristica', component_property='value')],
    [State(component_id='dd_trayectoria', component_property='value')]
)
def actualizar_grafico_perfiles(caracteristica, trayectoria):
    df = df_spain[df_spain.fpId.isin(trayectoria)]
    return px.scatter(data_frame = df, x = 'RTA', y = caracteristica, color = 'fpId') # timestamp
# line


@app.callback(
    Output(component_id='grafico_mapa', component_property='figure'),
    [Input(component_id='dd_tipo_mapa', component_property='value')],
    [State(component_id='dd_trayectoria', component_property='value')]
)
def actualizar_grafico_mapas(tipo_mapa, trayectoria):
    df = df_spain[df_spain.fpId.isin(trayectoria)].copy()
    df['minutos_vuelo'] = df.RTA // 60
    if tipo_mapa == '3d':
        return px.line_3d(data_frame = df, x = 'longitude', y = 'latitude', z = 'altitude', color = 'fpId',)
    elif tipo_mapa == 'geo':
        return px.line_geo(data_frame = df, lon = 'longitude', lat = 'latitude', projection='natural earth',
                          width=1200, height=700, color='fpId')
    elif tipo_mapa == 'mapbox':
        return px.scatter_mapbox(data_frame = df, lon = 'longitude', lat = 'latitude', zoom = 4, 
                              mapbox_style="carto-positron", hover_data = ['minutos_vuelo', 'altitude'],
                                width=1200, height=700, color = 'fpId', opacity = 0.5)
    


@app.callback(
    [Output(component_id='contenidos-tabs', component_property='children')],
    [Input(component_id='tabs', component_property='value'),
     Input(component_id='dd_trayectoria', component_property='value')]
)
def actualizar_tab_seleccionada(tab_seleccionada,trayectoria):
#     print(tab_seleccionada, trayectoria)
    if trayectoria is None:
        return (html.Div(),)
    else:
        if tab_seleccionada == 'visor':
            return (actualizar_visor(trayectoria),)
        elif tab_seleccionada == 'perfiles':
            return (actualizar_perfiles(trayectoria),)
        elif tab_seleccionada == 'mapas':
            return (actualizar_mapas(trayectoria),)

### 2.5. Funciones auxiliares para construir las pestañas

In [9]:
# Contenido de la pestaña Visor
def actualizar_visor(trayectoria):
    tray = df_spain[df_spain.fpId.isin(trayectoria)].sort_values(by=['fpId','timestamp'])
    return (html.H5('Visor de vectores de la trayectoria'),
            html.Table(children=[
                html.Tbody(children=[
                    html.Tr(children=[
                        html.Td('Trayectoria:'),
                        html.Td(trayectoria),
                        html.Td('Aeropuerto\norigen:'),
                        html.Td(tray.aerodromeOfDeparture.unique()[0])
                    ]),
                ])
            ], style=dict(marginTop=10)),
            dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in tray.columns[:11]],
                data=tray.to_dict('records'), # .loc[:,'timestamp':'hexident']
                page_size=50,
                sort_action='native',
                style_table=dict(overflowX='auto'),
                style_header=dict(fontWeight='bold', textAlign='center')
            ))

# Contenido de la pestaña Perfiles
def actualizar_perfiles(trayectoria):
    tray = df_spain[df_spain.fpId.isin(trayectoria)][['timestamp','altitude','speed','vspeed']]
    
    contenido = (
        html.H5('Perfiles temporales de la trayectoria'),
        dd_caracteristica,
        dcc.Graph(id='grafico_perfiles')
    )
    return contenido

# Contenido de la pestaña Trayectoria
def actualizar_mapas(trayectoria):
    tray = df_spain[df_spain.fpId.isin(trayectoria)][['timestamp','latitude','longitude','altitude']]
    
    contenido = (
        html.H5('Trayectoria 4D'),
        dd_tipo_mapa,
        dcc.Graph(id='grafico_mapa', 
                  style=dict(minHeight=550))
    )
    return contenido

### 2.6. Lanzamiento de la ejecución

In [10]:
app.run_server(debug = False)
# mode='inline'

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Jun/2022 14:15:49] "GET /_alive_8ea8c92e-ef0b-47c1-8fe6-de5e1d63a4f8 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


127.0.0.1 - - [13/Jun/2022 14:15:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:51] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:52] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:15:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:16:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:16:00] "POST /_

127.0.0.1 - - [13/Jun/2022 14:52:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:52:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:52:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:52:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:52:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:52:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Jun/2022 14:52:14] "POST /_dash-update-component HTTP/1.1" 200 -
